## 2-layer dense

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=45)

input_dims = (1,1,5,5,)
l1_output_dim = 100
l2_output_dim = 10

weights1 = (np.random.rand(l1_output_dim, np.product(input_dims[1:])) - 0.5) / 2
biases1 = (np.random.rand(l1_output_dim) - 0.5) / 2
weights2 = (np.random.rand(l2_output_dim, l1_output_dim) - 0.5) / 2
biases2 = (np.random.rand(l2_output_dim) - 0.5) / 2
inputs = np.random.rand(*input_dims) / 2

# note how the second layer does not have relu activation
pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(input_dims[1:]), out_features=l1_output_dim), 
    nn.ReLU(),
    nn.Linear(in_features=l1_output_dim, out_features=l2_output_dim),
)
pt_model[0].weight = nn.Parameter(torch.tensor(weights1))
pt_model[0].bias = nn.Parameter(torch.tensor(biases1))
pt_model[2].weight = nn.Parameter(torch.tensor(weights2))
pt_model[2].bias = nn.Parameter(torch.tensor(biases2))
pt_model(torch.tensor(inputs).reshape(input_dims[0],-1)).detach().numpy()

In [ ]:
t_max = 2**8
# we turn off relu activation in the SNN as well to get negative outputs
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Dense(weights=weights1, biases=biases1),
    layers.Dense(weights=weights2, biases=biases2, rectifying=False),
])

loihi_model(inputs, logging=False)

## 2-layer convolutional

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=45)

input_dims = (1,1,10,10)
conv_weight_dims1 = (6,1,8,8)
conv_weight_dims2 = (5,6,3,3)

conv_kernel_size1 = conv_weight_dims1[2:]
conv_kernel_size2 = conv_weight_dims2[2:]

weights1 = (np.random.rand(*conv_weight_dims1)-0.5) / 2
biases1 = (np.random.rand(conv_weight_dims1[0])-0.5) / 2
weights2 = (np.random.rand(*conv_weight_dims2)-0.5) / 2
biases2 = (np.random.rand(conv_weight_dims2[0])-0.5) / 2
inputs = np.random.rand(*input_dims) / 3

pt_model = nn.Sequential(
    nn.Conv2d(in_channels=conv_weight_dims1[1], out_channels=conv_weight_dims1[0], kernel_size=conv_kernel_size1), nn.ReLU(),
    nn.Conv2d(in_channels=conv_weight_dims2[1], out_channels=conv_weight_dims2[0], kernel_size=conv_kernel_size2), nn.ReLU(),
)
pt_model[0].weight = nn.Parameter(torch.tensor(weights1))
pt_model[0].bias = nn.Parameter(torch.tensor(biases1))
pt_model[2].weight = nn.Parameter(torch.tensor(weights2))
pt_model[2].bias = nn.Parameter(torch.tensor(biases2))
pt_model(torch.tensor(inputs)).detach().numpy()

In [ ]:
t_max = 2**7
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights1, biases=biases1),
    layers.Conv2D(weights=weights2, biases=biases2),
])

loihi_model(inputs)

## 2-layer maxpool

In [ ]:
import quartz
from quartz import layers
import numpy as np

input_dims = (1,1,8,8)
t_max = 2**7
kernel_size = [2,2]
np.random.seed(seed=27)
inputs = np.random.rand(*input_dims) / 2

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.MaxPool2D(kernel_size=kernel_size),
])
loihi_model(inputs)

In [ ]:
%debug

In [ ]:
loihi_model.layers[2].rectifier_neurons[0]


## 4-layer convolutional / maxpooling

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=44)

input_dims = (10,1,10,10)
weight_dims = ( 3,1,3,3)
weight_dims2 = (5,3,3,3)
pooling_kernel_size = [2,2]

weights1 = (np.random.rand(*weight_dims)-0.5) / 3
weights2 = (np.random.rand(*weight_dims2)-0.5) / 3
biases1 = (np.random.rand(weight_dims[0])-0.5) / 3
biases2 = (np.random.rand(weight_dims2[0])-0.5) / 3
inputs = np.random.rand(*input_dims) / 3

pt_model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=weight_dims[2:]), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size), nn.ReLU(),
    nn.Conv2d(in_channels=weight_dims2[1], out_channels=weight_dims2[0], kernel_size=weight_dims2[2:]), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size), nn.ReLU(),
)
pt_model[0].weight = nn.Parameter(torch.tensor(weights1))
pt_model[0].bias = nn.Parameter(torch.tensor(biases1))
pt_model[4].weight = nn.Parameter(torch.tensor(weights2))
pt_model[4].bias = nn.Parameter(torch.tensor(biases2))
model_output = pt_model(torch.tensor(inputs)).detach().numpy()

In [ ]:
t_max = 2**7
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights1, biases=biases1),
    layers.MaxPool2D(kernel_size=pooling_kernel_size),
    layers.Conv2D(weights=weights2, biases=biases2),
    layers.MaxPool2D(kernel_size=pooling_kernel_size),
])

loihi_model(inputs, logging=True)